In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as pyplot
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Load the .dat file (adjust delimiter as needed)
movies = pd.read_csv(r'C:\Users\user\Desktop\MICHELLE\Dev\Projects\movie-recommendation\datasets\movies.dat', sep='::', engine='python', encoding='ISO-8859-1') # Example with double colon delimiter


In [10]:
ratings = pd.read_csv(r'C:\Users\user\Desktop\MICHELLE\Dev\Projects\movie-recommendation\datasets\ratings.dat', sep='::', engine='python', encoding='ISO-8859-1')
users = pd.read_csv(r'C:\Users\user\Desktop\MICHELLE\Dev\Projects\movie-recommendation\datasets\users.dat', sep='::', engine='python', encoding='ISO-8859-1') 

In [11]:
movies.head()

,1,Toy Story (1995),Animation|Children's|Comedy
0,2,Jumanji (1995),Adventure|Children's|Fantasy
1,3,Grumpier Old Men (1995),Comedy|Romance
2,4,Waiting to Exhale (1995),Comedy|Drama
3,5,Father of the Bride Part II (1995),Comedy
4,6,Heat (1995),Action|Crime|Thriller


In [12]:
ratings.head()

,1,1193,5,978300760
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1197,3,978302268


In [13]:
users.head()

,1,F,1.1,10,48067
0,2,M,56,16,70072
1,3,M,25,15,55117
2,4,M,45,7,02460
3,5,M,25,20,55455
4,6,F,50,9,55117


In [14]:
# Specify column names for each dataset
user_columns = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']
movie_columns = ['MovieID', 'Title', 'Genres']
rating_columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']

# Load the datasets with specified column names
users = pd.read_csv('datasets/users.dat', sep='::', engine='python', encoding='ISO-8859-1', names=user_columns)
movies = pd.read_csv('datasets/movies.dat', sep='::', engine='python', encoding='ISO-8859-1', names=movie_columns)
ratings = pd.read_csv('datasets/ratings.dat', sep='::', engine='python', encoding='ISO-8859-1', names=rating_columns)


In [15]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
